# Debug error

2023.03.13

In [1]:
using JuMP, Ipopt

In [2]:
include("../src/NLPSaUT.jl")

Main.NLPSaUT

In [28]:
function fitness(x...)
    fval = x[1]^2 + x[2]
    h = x[1]^3 + x[2] - 1
    #g = [1]^2 + x[2]^2 - 1
    return vcat(fval,h)[:]
end

fitness (generic function with 1 method)

In [29]:
nx = 2
nfitness = 2

2

In [30]:
memoized_fitness = NLPSaUT.memoize_fitness(fitness, nx, nfitness)

2-element Vector{Main.NLPSaUT.var"#2#5"{Int64, Main.NLPSaUT.var"#f_fitness_i#3"{typeof(fitness)}}}:
 #2 (generic function with 1 method)
 #2 (generic function with 1 method)

In [31]:
memoized_fitness[1]([0.1,0.2]...)

0.21000000000000002

In [32]:
∇memoized_fitness = NLPSaUT.memoize_fitness_gradient(
    fitness, nx, nfitness, 2
)

2-element Vector{Main.NLPSaUT.var"#7#15"{Int64, Main.NLPSaUT.var"#foo_fwd#13"{Int64, Main.NLPSaUT.var"#f_fitness_nosplat#12"{typeof(fitness)}}}}:
 #7 (generic function with 1 method)
 #7 (generic function with 1 method)

In [33]:
model = Model(Ipopt.Optimizer)

lx = [-1,-1]
ux = [ 1, 1]
x0 = [0.2, 0.3]
@variable(model, lx[i] <= x[i=eachindex(lx)] <= ux[i], start=x0[i])

2-element Vector{VariableRef}:
 x[1]
 x[2]

In [34]:
#register(model, :f, nx, memoized_fitness[1], ∇memoized_fitness[1])
#register(model, :f, nx, memoized_fitness[1]; autodiff = true)
register(model, :f2, nx, memoized_fitness[1]; autodiff = true)

In [35]:
#@NLobjective(model, Min, f(x[1], x[2]))   # works

In [36]:
@NLobjective(model, Min, f2(x...))   # doesn't work -> now works

In [37]:
optimize!(model)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.4000000e-01 0.00e+00 1.00e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00  

### Memoization example

https://jump.dev/JuMP.jl/stable/tutorials/nonlinear/tips_and_tricks/#User-defined-functions-with-vector-outputs

In [29]:
function_calls = 0
function foo(z...)
    x = z[1]
    y = z[2]
    global function_calls += 1
    common_term = x^2 + y^2
    term_1 = sqrt(1 + common_term)
    term_2 = common_term
    return term_1, term_2
end

foo (generic function with 1 method)

In [68]:
memoized_foo = NLPSaUT.memoize_fitness(foo, 2, 2)

2-element Vector{Main.NLPSaUT.var"#2#5"{Int64, Main.NLPSaUT.var"#f_fitness_i#3"{typeof(foo)}}}:
 #2 (generic function with 1 method)
 #2 (generic function with 1 method)

In [69]:
function_calls = 0
memoized_foo[1](1.0, 1.0)
println("function_calls = ", function_calls)

function_calls = 1


In [70]:
model = Model(Ipopt.Optimizer)
#set_silent(model)
#@variable(model, x[1:2] >= 0, start = 0.1)

lx = [-1,-1]
ux = [ 1, 1]
x0 = [0.2, 0.3]
@variable(model, lx[i] <= x[i=keys(lx)] <= ux[i], start=x0[i])

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [1, 2]
And data, a 2-element Vector{VariableRef}:
 x[1]
 x[2]

In [71]:
register(model, :foo_1, 2, memoized_foo[1]; autodiff = true)
register(model, :foo_2, 2, memoized_foo[2]; autodiff = true)
@NLobjective(model, Max, foo_1(x[1], x[2]))
@NLconstraint(model, foo_2(x[1], x[2]) <= 2)
function_calls = 0
optimize!(model)
@show objective_value(model) 
println("Memoized approach: function_calls = $(function_calls)")

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0630146e+00 0.00e+00 2.09e-01   0.0 0.00e+00    -  0.00e+00 0.00e+00  